In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [ ]:
import pandas as pd
import requests
import os
import gzip
import json
import numpy as np

import pprint
import tempfile
from string import printable as pt

from typing import Dict, Text

import tensorflow as tf
# from tfx import v1 as tfx

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
ratings_df = pd.read_csv('/content/drive/MyDrive/KaggleX/model_data/ratings/all_ratings.csv')

In [ ]:
books_df = pd.read_csv('/content/drive/MyDrive/KaggleX/model_data/books/all_books.csv')

In [ ]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285451 entries, 0 to 285450
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            285451 non-null  int64  
 1   isbn                  198124 non-null  object 
 2   text_reviews_count    285377 non-null  float64
 3   series                285451 non-null  object 
 4   country_code          285379 non-null  object 
 5   language_code         192530 non-null  object 
 6   popular_shelves       285451 non-null  object 
 7   asin                  51293 non-null   object 
 8   is_ebook              285379 non-null  object 
 9   average_rating        285377 non-null  float64
 10  kindle_asin           176875 non-null  object 
 11  similar_books         285451 non-null  object 
 12  description           258490 non-null  object 
 13  format                217552 non-null  object 
 14  link                  285377 non-null  object 
 15  

In [ ]:
books_df.head()

,Unnamed: 0,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0,0743294297,3282.0,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",NaN,False,3.49,...,7.0,NaN,2009.0,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,6243154.0,Best Friends Forever,Best Friends Forever
1,1,1934876569,6.0,['151854'],US,NaN,"[{'count': '515', 'name': 'to-read'}, {'count'...",NaN,False,4.22,...,3.0,NaN,2009.0,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98.0,701117.0,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...
2,2,0922915113,39.0,[],US,NaN,"[{'count': '961', 'name': 'to-read'}, {'count'...",NaN,False,3.81,...,4.0,NaN,2000.0,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,287149,986.0,278586.0,The Devil's Notebook,The Devil's Notebook
3,3,NaN,60.0,['1052227'],US,eng,"[{'count': '54', 'name': 'currently-reading'},...",B01NCIKAQX,True,4.33,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,33394837,269.0,54143148.0,The House of Memory (Pluto's Snitch #2),The House of Memory (Pluto's Snitch #2)
4,4,0800759494,2885.0,[],US,NaN,"[{'count': '9381', 'name': 'to-read'}, {'count...",NaN,False,3.91,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,89375,68157.0,2957021.0,90 Minutes in Heaven: A True Story of Death an...,90 Minutes in Heaven: A True Story of Death an...


In [ ]:
import ast

books_df['authors'][0]
books_df_parsed_author = books_df.copy()
books_df_parsed_author['authors'] = books_df['authors'].apply(lambda x: ast.literal_eval(x))
books_df_parsed_author['authors'] = books_df_parsed_author['authors'].apply(lambda x: ', '.join([i['author_id'] for i in x]))



In [ ]:
books_df_parsed_tags = books_df_parsed_author.copy()
books_df_parsed_tags['popular_shelves'] = books_df_parsed_author['popular_shelves'].apply(lambda x: ast.literal_eval(x))
books_df_parsed_tags['popular_shelves']  = books_df_parsed_tags['popular_shelves'] .apply(lambda x: ', '.join([i['name'] for i in x][:10]))


In [ ]:
# books_df_parsed_tags['popular_shelves']

In [ ]:
books_df = books_df_parsed_tags.copy()
# books_df = books_df_parsed_author.copy()

In [ ]:
# ratings_df.fillna('', inplace=True)
books_df.fillna('', inplace=True)

allowed_set = set(pt)
# ratings_df['description'] = ratings_df['description'].apply(lambda x:  ''.join([' ' if  s not in  allowed_set else s for s in x]))
books_df['description'] = books_df['description'].apply(lambda x:  ''.join([' ' if  s not in  allowed_set else s for s in x]))
books_df['authors'] = books_df['authors'].apply(lambda x:  ''.join([' ' if  s not in  allowed_set else s for s in x]))
books_df['popular_shelves'] = books_df['popular_shelves'].apply(lambda x:  ''.join([' ' if  s not in  allowed_set else s for s in x]))


books_df.description.replace({r'[^a-zA-Z0-9 ]+':''}, regex=True, inplace=True)
books_df.authors.replace({r'[^a-zA-Z0-9 ]+':''}, regex=True, inplace=True)
books_df.popular_shelves.replace({r'[^a-zA-Z0-9 ]+':''}, regex=True, inplace=True)
# ratings_df.description.replace({r'[^a-zA-Z0-9 ]+':''}, regex=True, inplace=True)

# ratings_df=ratings_df.replace(r'^\s*$', np.nan, regex=True)
books_df=books_df.replace(r'^\s*$', np.nan, regex=True)

# ratings_df['description']=ratings_df['description'].fillna('None')
books_df['description']=books_df['description'].fillna('None')
books_df['authors']=books_df['authors'].fillna('None')
books_df['popular_shelves']=books_df['authors'].fillna('None')
# ds=ds2
# ds_song=ds.groupby(['song_id','title','release','artist_name','year']).size().reset_index().rename(columns={0:'count'})

In [ ]:
ratings_df = ratings_df.merge(books_df, how='inner', on='book_id')

In [ ]:
books_df = books_df[['title', 'popular_shelves', 'description', 'authors']]

In [ ]:
ratings_df = ratings_df[['user_id', 'title', 'popular_shelves', 'description', 'authors']]

In [ ]:
print(ratings_df['description'].isnull().sum())

0


In [ ]:
books_df.head()

,title,popular_shelves,description,authors
0,Best Friends Forever,9212,Addie Downs and Valerie Adler were eight when ...,9212
1,All's Fairy in Love and War (Avalon: Web of Ma...,19158,To Karas astonishment she discovers that a por...,19158
2,The Devil's Notebook,2983296 40075,Wisdom humor and dark observations by the foun...,2983296 40075
3,The House of Memory (Pluto's Snitch #2),242185,None,242185
4,90 Minutes in Heaven: A True Story of Death an...,39054 11099,As he is driving home from a ministers confere...,39054 11099


In [ ]:
ratings = tf.data.Dataset.from_tensor_slices((dict(ratings_df)))
books = tf.data.Dataset.from_tensor_slices((dict(books_df)))

In [ ]:
ratings = ratings.map(lambda x: {
    "title": x["title"],
    "authors": x["authors"],
    "popular_shelves": x["popular_shelves"],
    "description": x["description"],
    "user_id": x["user_id"],

})

books = books.map(lambda x: {
      "title": x["title"],
      "authors": x["authors"],
      "popular_shelves": x["popular_shelves"],
      "description": x["description"],
})

books_titles = books.map(lambda x: x["title"])
# book_descriptions = books.map(lambda x: x["description"])
book_descriptions = books.map(lambda x: x["description"])
book_authors = books.map(lambda x: x["authors"])
book_popular_shelves = books.map(lambda x: x["popular_shelves"])

In [ ]:
# unique_book_titles = np.unique(np.concatenate(list(books.batch(1000))))

unique_book_titles = np.unique(np.concatenate(list(books.batch(1_000).map(
    lambda x: x["title"]))))

unique_authors = np.unique(np.concatenate(list(books.batch(1_000).map(
    lambda x: x["authors"]))))

unique_popular_shelves = np.unique(np.concatenate(list(books.batch(1_000).map(
    lambda x: x["popular_shelves"]))))

unique_descriptions = np.unique(np.concatenate(list(books.batch(1_000).map(
    lambda x: x["description"]))))



unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

In [ ]:
embedding_dimension = 32

In [ ]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()


    self.user_id_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])


  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_id_embedding(inputs["user_id"]),
    ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
# title_text = tf.keras.layers.TextVectorization()
# title_text.adapt(ratings.map(lambda x: x["popular_shelves"]))


In [ ]:
# for row in ratings.batch(1).map(lambda x: x["popular_shelves"]).take(1):
#   print(title_text(row))

In [ ]:
# title_text.get_vocabulary()[15:45]

In [ ]:
class BookModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_book_titles,mask_token=None),
      tf.keras.layers.Embedding(len(unique_book_titles) + 1, embedding_dimension)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)


    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(unique_book_titles)


    self.description_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)


    self.description_text_embedding = tf.keras.Sequential([
      self.description_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.description_vectorizer.adapt(unique_descriptions)


    self.author_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)


    self.author_text_embedding = tf.keras.Sequential([
      self.author_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.author_vectorizer.adapt(unique_authors)


    self.popular_shelves_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)


    self.popular_shelves_text_embedding = tf.keras.Sequential([
      self.popular_shelves_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.popular_shelves_vectorizer.adapt(unique_popular_shelves)



    # self.genre_text_embedding = tf.keras.Sequential([
    #   tf.keras.layers.TextVectorization(max_tokens=max_tokens),
    #   tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
    #   tf.keras.layers.GlobalAveragePooling1D(),
    # ])

    # self.author_text_embedding = tf.keras.Sequential([
    #   tf.keras.layers.TextVectorization(max_tokens=max_tokens),
    #   tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
    #   # We average the embedding of individual words to get one embedding vector
    #   # per title.
    #   tf.keras.layers.GlobalAveragePooling1D(),
    # ])

  def call(self, inputs):
    return tf.concat([
        self.title_embedding(inputs["title"]),
        self.title_text_embedding(inputs["title"]),
        self.description_text_embedding(inputs["description"]),
        self.popular_shelves_text_embedding(inputs["popular_shelves"]),
        self.author_text_embedding(inputs["authors"]),

    ], axis=1)


In [ ]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = BookModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class GoodreadsModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=books.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):

    query_embeddings = self.query_model({
        "user_id": features["user_id"],
    })
    book_embeddings = self.candidate_model({
        "title":features["title"],
        "description":features["description"],
        "popular_shelves":features["popular_shelves"],
        "authors":features["authors"],
        })

    return self.task(
        query_embeddings, book_embeddings, compute_metrics=not training)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(800_000)
test = shuffled.skip(800_000).take(200_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [ ]:
num_epochs = 100

model = GoodreadsModel([64, 32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))


In [ ]:
one_layer_history = model.fit(
    cached_train,
    epochs=300)

# accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
# print(f"Top-100 accuracy: {accuracy:.2f}.")

Epoch 1/300
391/391 [==============================] - 96s 240ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 14173.9209 - regularization_loss: 0.0000e+00 - total_loss: 14173.9209
Epoch 2/300
391/391 [==============================] - 94s 239ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 13131.1274 - regularization_loss: 0.0000e+00 - total_loss: 13131.1274
Epoch 3/300
391/391 [==============================] - 92s 232ms/step - factorized_top_k/top_1_categorical_accuracy:

In [ ]:
model.candidate_model.save("/content/drive/MyDrive/KaggleX/models/candidate_model2", save_format='tf')
loaded_model = tf.keras.saving.load_model("/content/drive/MyDrive/KaggleX/models/candidate_model2")

In [ ]:
model.evaluate(cached_test, return_dict=True)

49/49 [==============================] - 1672s 34s/step - factorized_top_k/top_1_categorical_accuracy: 1.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 9.0500e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0016 - factorized_top_k/top_50_categorical_accuracy: 0.0054 - factorized_top_k/top_100_categorical_accuracy: 0.0100 - loss: 53581.1717 - regularization_loss: 0.0000e+00 - total_loss: 53581.1717


{'factorized_top_k/top_1_categorical_accuracy': 0.00013000000035390258,
 'factorized_top_k/top_5_categorical_accuracy': 0.0009050000226125121,
 'factorized_top_k/top_10_categorical_accuracy': 0.0016049999976530671,
 'factorized_top_k/top_50_categorical_accuracy': 0.005365000106394291,
 'factorized_top_k/top_100_categorical_accuracy': 0.009999999776482582,
 'loss': 44423.7734375,
 'regularization_loss': 0,
 'total_loss': 44423.7734375}

In [ ]:
model.candidate_model.summary()

In [ ]:
# model.candidate_model.predict(["The Three-Body Problem Remembrance of Earth's Past #1"])

model.candidate_model.predict({ "title": np.array(["The Three-Body Problem Remembrance of Earth's Past #1"]),
                           "description":np.array(["The Three-Body Problem Remembrance of Earth's Past #1 description"]),
                           "authors":np.array(['1234,5678']),
                                "popular_shelves":np.array(['to-read,fantasy'])})

1/1 [==============================] - 0s 415ms/step


array([[-2.003811  ,  1.396957  ,  1.1332344 ,  1.4173847 , -0.59739983,
         0.6107756 ,  1.3834804 ,  0.03517982,  0.8437845 , -1.4226427 ,
         1.0381627 ,  1.7822137 , -0.7536729 ,  2.0567286 ,  0.4008212 ,
        -0.5157179 ,  0.77249897,  1.7593852 , -0.4889506 , -0.8746738 ,
         0.13983527, -0.3523752 , -1.1248167 ,  0.80105066, -1.7619702 ,
        -0.20873323, -0.30861586,  0.4628358 , -0.0109731 , -0.45345497,
         0.08961838,  1.2657    ]], dtype=float32)

In [ ]:
# model.build()